In [1]:
TANGO_HOST="192.168.128.118:10000"
correlator_setup = 'two_receptor'
visibility_ip = "10.50.1.33"
dish_ids = ["SKA001"]
correlator_setup = "1fsp"

In [16]:
from PyTango import DeviceProxy, AttributeProxy
import os
import json
from time import sleep
from datetime import datetime

os.environ["TANGO_HOST"] = TANGO_HOST

csp_controller ="mid-csp/control/0"
csp_server = "mid-csp/subarray/01"
csp_controller_dp = DeviceProxy(csp_controller)
csp_dp = DeviceProxy(csp_server)

csp_controller_dp.adminMode = 0

with open(f'../../data/mid_telescope/cbf/sys_params/load_dish_config.json', 'r') as json_data:
    d = json.load(json_data)
    csp_controller_dp.LoadDishCfg(json.dumps(d))

csp_controller_dp.cbfSimulationMode = 0
csp_controller_dp.commandTimeout = 100
csp_controller_dp.On([])

In [7]:

print(csp_dp.obsState)
with open(f'../../data/mid_telescope/csplmc/assign_resources.json', 'r') as json_data:
    d = json.load(json_data)
    d["dish"]["receptor_ids"] = dish_ids
    print(d)
    csp_dp.AssignResources(json.dumps(d))

sleep(3)

print(csp_dp.obsState)

with open(f'../../data/mid_telescope/csplmc/configure_scan.json', 'r') as json_data:
    d = json.load(json_data)
    d["cbf"]["fsp"][0]["output_host"][0][1] = visibility_ip
    print(d)
    csp_dp.configure(json.dumps(d))

sleep(3)
print(csp_dp.obsState)
sleep(5)
print(csp_dp.obsState)

obsState.FAULT
{'interface': 'https://schema.skao.int/ska-csp-assignresources/2.2', 'subarray_id': 1, 'dish': {'receptor_ids': ['SKA001']}}


DevFailed: DevFailed[
DevError[
    desc = ska_tango_base.faults.StateModelError: AssignResources command not permitted in observation state FAULT
           
  origin = Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/tango/device_server.py", line 85, in wrapper
    return get_worker().execute(fn, *args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/tango/green.py", line 101, in execute
    return fn(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/ska_tango_base/subarray/subarray_device.py", line 574, in is_AssignResources_allowed
    raise StateModelError(
ska_tango_base.faults.StateModelError: AssignResources command not permitted in observation state FAULT

  reason = PyDs_PythonError
severity = ERR]

DevError[
    desc = Failed to execute command_inout on device mid-csp/subarray/01, command AssignResources
  origin = virtual Tango::DeviceData Tango::Connection::command_inout(const string&, const Tango::DeviceData&) at 

In [15]:
print(csp_dp.obsState)

obsState.FAULT


VISIBILITY POD

In [ ]:
now = datetime.now()

with open(f'../../data/mid_telescope/csplmc/delay_model.json', 'r') as json_data:
    delay_model = json.load(json_data)

delay_model["epoch"] = float(int(now.timestamp()))

print(delay_model)

tm_server = "ska_mid/tm_leaf_node/csp_subarray_01"
tm_dp = DeviceProxy(tm_server)
tm_dp.delayModel = json.dumps(delay_model)

sleep(5)

with open(f'../../data/scan.json', 'r') as json_data:
    d = json.load(json_data)
    csp_dp.scan(json.dumps(d))

sleep(2)
print(csp_dp.obsState)

In [ ]:
csp_dp.EndScan()

In [5]:
csp_dp.GoToIdle()
sleep(2)
csp_dp.ReleaseAllResources()

DevFailed: DevFailed[
DevError[
    desc = ska_tango_base.faults.StateModelError: GoToIdle command not permitted in observation stateFAULT
           
  origin = Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/tango/device_server.py", line 85, in wrapper
    return get_worker().execute(fn, *args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/tango/green.py", line 101, in execute
    return fn(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/ska_csp_lmc_common/subarray_device.py", line 546, in is_GoToIdle_allowed
    raise StateModelError(
ska_tango_base.faults.StateModelError: GoToIdle command not permitted in observation stateFAULT

  reason = PyDs_PythonError
severity = ERR]

DevError[
    desc = Failed to execute command_inout on device mid-csp/subarray/01, command GoToIdle
  origin = virtual Tango::DeviceData Tango::Connection::command_inout(const string&, const Tango::DeviceData&) at (/src/cppTango/cppapi/client/devapi

In [8]:
csp_dp.ObsReset()

[array([2], dtype=int32), ['1713304442.9438663_222374179023306_ObsReset']]